<a href="https://colab.research.google.com/github/gabikohen/Ejercicio-PCA/blob/master/Ejercicio_de_PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Carga los datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# load the dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
# view the shape of the dataset
mnist.data.shape

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(70000, 784)

2. Prepara los datos

In [ ]:
X = mnist.data
y = mnist.target

Train y test

Prepara los datos

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Escalar los datos.

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Aplicar PCA para retener el 95% de la varianza

In [ ]:
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

Ver las dimensiones antes y después de aplicar ACP.

In [ ]:
print("Dimensiones antes de ACP:", X_train_scaled.shape)
print("Dimensiones después de ACP:", X_train_pca.shape)

Dimensiones antes de ACP: (49000, 784)
Dimensiones después de ACP: (49000, 327)


3. Crea 2 modelos KNN

a. Uno que utilice los datos transformados con ACP para predecir qué número muestra cada imagen.

In [ ]:
knn_pca = KNeighborsClassifier()
knn_pca.fit(X_train_pca, y_train)

train_preds_knn_pca = knn_pca.predict(X_train_pca)
test_preds_knn_pca = knn_pca.predict(X_test_pca)

train_score_knn_pca = accuracy_score(y_train, train_preds_knn_pca)
test_score_knn_pca = accuracy_score(y_test, test_preds_knn_pca)
print (f"El accuracy del conjunto de train es {train_score_knn_pca:.3f}.")
print (f"El accuracy del conjunto de test es {test_score_knn_pca:.3f}.")

El accuracy del conjunto de train es 0.964.
El accuracy del conjunto de test es 0.947.


b. Uno que utilice los datos originales sin la transformación en ACP (pero recuerda que todavía tienes que escalar los datos).

In [ ]:
knn_ori = KNeighborsClassifier()
knn_ori.fit(X_train_scaled, y_train)

train_preds_knn_ori = knn_ori.predict(X_train_scaled)
test_preds_knn_ori = knn_ori.predict(X_test_scaled)

train_score_knn_ori = accuracy_score(y_train, train_preds_knn_ori)
test_score_knn_ori = accuracy_score(y_test, test_preds_knn_ori)
print (f"El accuracy del conjunto de train es {train_score_knn_ori:.3f}.")
print (f"El accuracy del conjunto de test es {test_score_knn_ori:.3f}.")

El accuracy del conjunto de train es 0.962.
El accuracy del conjunto de test es 0.943.


# **4. Evalúa y compara los modelos.**

Evaluar modelo que utiliza los datos transformados mediante ACP.

In [ ]:
%%time
print("Matriz de Confusión (ACP):\n", confusion_matrix(y_test, test_preds_knn_pca))
print("\nReporte de Clasificación (ACP):\n", classification_report(y_test, test_preds_knn_pca))

Matriz de Confusión (ACP):
 [[2024    1    5    2    0    7   16    2    1    0]
 [   0 2342    9    2    3    0    2    3    1    2]
 [  20   24 2004   25    8    6   16   12   13    5]
 [   5    4   24 2047    2   27    2   29   20   16]
 [   0   19   19    2 1822    4    6    7    2   55]
 [   8    2    2   54    9 1785   27    3   15   10]
 [  17    3    6    0    8   12 2038    0    4    0]
 [   3   25   10    7   23    1    0 2104    2   73]
 [  21   25   19   29   11   57   11   13 1786   20]
 [  10    6   14   17   36    4    0   57    6 1940]]

Reporte de Clasificación (ACP):
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      2058
           1       0.96      0.99      0.97      2364
           2       0.95      0.94      0.94      2133
           3       0.94      0.94      0.94      2176
           4       0.95      0.94      0.94      1936
           5       0.94      0.93      0.94      1915
           6       0.96     

Evaluar modelo que utiliza los datos originales sin la transformación en ACP.

In [ ]:
%%time
print("Matriz de Confusión (original):\n", confusion_matrix(y_test, test_preds_knn_ori))
print("\nReporte de Clasificación (original):\n", classification_report(y_test, test_preds_knn_ori))

Matriz de Confusión (original):
 [[2023    1    6    2    0    7   16    2    1    0]
 [   0 2342   11    2    3    0    2    2    1    1]
 [  24   26 1991   31    7    7   17   15   10    5]
 [   4    5   27 2050    2   23    2   28   19   16]
 [   0   23   19    3 1800    2    6   11    3   69]
 [  11    3    2   54   12 1784   25    2   11   11]
 [  17    6    7    0   11   13 2031    0    3    0]
 [   3   34   10    5   29    1    0 2089    1   76]
 [  17   31   17   39   11   66   10   16 1767   18]
 [  10    7   13   21   42    4    0   62    5 1926]]

Reporte de Clasificación (original):
               precision    recall  f1-score   support

           0       0.96      0.98      0.97      2058
           1       0.95      0.99      0.97      2364
           2       0.95      0.93      0.94      2133
           3       0.93      0.94      0.94      2176
           4       0.94      0.93      0.93      1936
           5       0.94      0.93      0.93      1915
           6      

# 5. Responde las siguientes preguntas en un texto:**bold text**

a. ¿Qué modelo funcionó mejor en el conjunto de prueba?

La evaluación de los modelos revela que el rendimiento superior se observa en el modelo que utiliza datos transformados mediante ACP. Este modelo exhibió puntajes más altos en métricas clave, como precisión, recall y F1-score, en comparación con el modelo que emplea datos originales. Estos resultados sugieren que la reducción de dimensionalidad proporcionada por ACP contribuyó positivamente a la capacidad predictiva del modelo.

b. ¿Qué modelo fue el más rápido en hacer predicciones?

En cuanto a la velocidad de predicción, se observa que el modelo que utiliza los datos originales muestra tiempos de ejecución más cortos, indicando una mayor eficiencia en la realización de predicciones en comparación con el modelo que utiliza datos transformados mediante ACP. Esto sugiere que, a pesar de la mejora en el rendimiento del modelo con ACP, el modelo con datos originales presenta una ventaja en términos de rapidez en la generación de predicciones.